In [4]:
from google.colab import drive 
from google.colab import files

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [5]:
!pip install gensim==3.8.3 --quiet

!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install pydot --quiet
!pip install -q sentencepiece
# !pip install -q transformers
# !pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0
!pip install transformers
!pip install pytorch_lightning
# !pip install rouge-score

     |████████████████████████████████| 24.2 MB 1.2 MB/s 
     |████████████████████████████████| 4.9 MB 31.4 MB/s 
     |████████████████████████████████| 1.2 MB 27.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.6 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 27.3 MB/s 
     |████████████████████████████████| 141 kB 74.9 MB/s 
     |████████████████████████████████| 419 kB 54.3 MB/s 


# Imports

In [6]:

import numpy as np
import pandas as pd
from collections import Counter, OrderedDict

import pickle
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text


import sklearn as sk
import os
import nltk
from nltk.corpus import reuters
from nltk.data import find

import matplotlib.pyplot as plt

import re
import yaml

#This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.  
#Make sure your pip install command specifies gensim==3.8.3
import gensim

import numpy as np

from transformers import BertTokenizer, TFBertModel, T5Tokenizer, TFT5ForConditionalGeneration
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
from torchmetrics.text.rouge import ROUGEScore


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pprint import pprint

from pathlib import Path
import logging
import time
from logging.handlers import RotatingFileHandler



import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
import math

# import nltk
# nltk.download('punkt')
# from nltk.tokenize import sent_tokenize
import transformers
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchmetrics
import tensorboard
from tensorboard.backend.event_processing import event_accumulator



# device = 'cuda' if cuda.is_available() else 'cpu'
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers



from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,  BartTokenizer, BartForConditionalGeneration,  BartConfig
)


import gc
import string
import re
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction




In [37]:
pd.read_csv("./gdrive/MyDrive/W266 Final Project 2/data/new_data/test.csv")

,song_id,inputs,targets,inputs_b,targets_b
0,Beyonce-and-kendrick-lamar-nile-lyrics,"\n[Verse 1: Beyoncé]\nHey little buddy, where ...",<extra_id_0>\n[Chorus: Kendrick Lamar & Beyonc...,"\n[Verse 1: Beyoncé]\nHey little buddy, where ...",<mask>\n[Chorus: Kendrick Lamar & Beyoncé]\nOn...
1,Mihty-jeremih-and-ty-dolla-sign-the-light-lyrics,\n[Intro]\nHol' up!\nHitmaka!\n[Refrain: Ty Do...,"<extra_id_0>\n[Chorus: Jeremih]\nOoh, it's the...",\n[Intro]\nHol' up!\nHitmaka!\n[Refrain: Ty Do...,"<mask>\n[Chorus: Jeremih]\nOoh, it's the way i..."
2,Mihty-jeremih-and-ty-dolla-sign-the-light-lyrics,\n[Intro]\nHol' up!\nHitmaka!\n[Refrain: Ty Do...,"<extra_id_0>\n[Chorus: Jeremih]\nOoh, it's the...",\n[Intro]\nHol' up!\nHitmaka!\n[Refrain: Ty Do...,"<mask>\n[Chorus: Jeremih]\nOoh, it's the way i..."
3,Mihty-jeremih-and-ty-dolla-sign-the-light-lyrics,\n[Intro]\nHol' up!\nHitmaka!\n[Refrain: Ty Do...,"<extra_id_0>\n[Chorus: Jeremih]\nOoh, it's the...",\n[Intro]\nHol' up!\nHitmaka!\n[Refrain: Ty Do...,"<mask>\n[Chorus: Jeremih]\nOoh, it's the way i..."
4,Frank-ocean-swim-good-lyrics,\n[Verse 1]\nThat's a pretty big trunk on my L...,<extra_id_0>\n[Pre-Chorus]\nAnd I've got this ...,\n[Verse 1]\nThat's a pretty big trunk on my L...,<mask>\n[Pre-Chorus]\nAnd I've got this black ...
...,...,...,...,...,...
3297,Meek-mill-maybach-curtains-lyrics,\n[Produced by The Agency and Infamous]\n<extr...,<extra_id_0>\n[Hook: John Legend & Meek Mill]\...,\n[Produced by The Agency and Infamous]\n<mask>,<mask>\n[Hook: John Legend & Meek Mill]\nLook ...
3298,Meek-mill-maybach-curtains-lyrics,\n[Produced by The Agency and Infamous]\n[Vers...,<extra_id_0>\n[Hook: John Legend]\nLook how fa...,\n[Produced by The Agency and Infamous]\n[Vers...,<mask>\n[Hook: John Legend]\nLook how far we c...
3299,Meek-mill-maybach-curtains-lyrics,\n[Produced by The Agency and Infamous]\n[Vers...,<extra_id_0>\n[Hook: John Legend & Rick Ross]\...,\n[Produced by The Agency and Infamous]\n[Vers...,<mask>\n[Hook: John Legend & Rick Ross]\nLook ...
3300,Meek-mill-maybach-curtains-lyrics,\n[Produced by The Agency and Infamous]\n[Vers...,<extra_id_0>\n[Verse 3: Rick Ross]\nThe chorus...,\n[Produced by The Agency and Infamous]\n[Vers...,<mask>\n[Verse 3: Rick Ross]\nThe chorus cinem...


In [ ]:
# find index of "chorus" or "hook" and iterate till you hit 1st double new line .

# then iterate through number of counts of chorus and replace it with <extra_id_i>. Get tuple of start and end indices of each chorus.

# This will help us find start and end indices of each verse

 

In [7]:
# then clean strings 
# remove chorus and verse titles

# keep new lines in there. Just remove double new lines
# keep punctuation in there for now

# clean the text
def preprocess(sentence):
    sentence=str(sentence)

    #uncased means words are all lowercased
    sentence = sentence.lower()

    # remove chorus and verse titles
    # remove the brackets and everything in the brackets
    sentence = re.sub("[\[].*?[\]]", "", sentence)
   
    # replace parentheses only
    sentence = re.sub("[\(\)]", "", sentence)

    sentence = sentence.replace("\n", " ")

    # remove double new lines with single new line. So there is a new line space between sections
    # sentence= re.sub("(\n\n)", "\n", sentence)

    return sentence


# p = preprocess(verse_masked_lyrics)
# print(p)

# cleantrain_texts = list(map(preprocess, train_texts))

# #you need to make sure you apply the same preprocessing to the test and validation sets

# ### YOUR CODE HERE

# cleantest_texts = list(map(preprocess, test_texts))
# cleanvalid_texts = list(map(preprocess, valid_texts))
# ### END YOUR CODE

# BART 

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/t5_fine_tuning.ipynb#scrollTo=HS8mNXq6bdxq

https://github.com/Lightning-AI/lightning/discussions/11770

https://shivanandroy.com/fine-tune-t5-transformer-with-pytorch/


In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
class BartFineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(BartFineTuner, self).__init__()
    # self.hparams = hparams
    self.hparams.update(vars(hparams))
    self.save_hyperparameters()
    
    self.model = BartForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = BartTokenizer.from_pretrained(hparams.tokenizer_name_or_path, model_max_length = hparams.max_seq_length )
    self.acc = torchmetrics.Accuracy(mdmc_average = 'samplewise')
    self.scorer = ROUGEScore() #rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      # doesn't have a max_length method, so it will keep trying to predict the same length as the target ids
      self, input_ids, attention_mask=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        # decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        # takes in labels and shifts right to create decoder input ids shift
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    # print("Output keys: ", outputs.keys())
    # print("Batch: " , batch)
    # print("Batch shape :" , batch.shape)


    # source = batch['source_ids']

    #  Pred Output size :  torch.Size([8, 256, 32128])
    # Target Labels size :  torch.Size([8, 256])


    # values are the probabilities.
    # indices are the indices
    # y_pred = y_max_idx  # predictions are really the inx \in [n_classes] with the highest scores
    
    


  
    
    # print("******")
    # print("Pred Output size : ", preds.size())
    # print("Target Labels size : ", torch.max(labels, torch.tensor([0])).size())


    pred_ids = torch.max(outputs['logits'], 2).indices

    # change pred ids index of blank string token at end to index of PAD token, like target label has
    # go backward
    # DON"T NEED FOR BART
    # for i in range(pred_ids.size(0)):
    #   for j in range(1, pred_ids.size(1) + 1):
    #     # if  value is  3, then replace current valuw ith 0 and break
    #     # print(k[i][j])
    #     if pred_ids[i][-j] == 3:
    #       pred_ids[i][-j]=0
    #     else:
    #       break
  

    # change all the empty strings (word ind = 3) at the end to 0's to represent PAD token ind.
    # for each tensor in pred_ids, maks only 

    pred_str_li = self.tokenizer.batch_decode(pred_ids,skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    # get the values before the cast. set -100 to 1 . Pad token or 1 is the same, it will be ignored in string
    target_ids = torch.max(batch["target_ids"], torch.tensor([1]).cuda())
    # print("target_ids : ", target_ids)
    # print("pred_ids : ", pred_ids)



    # print("Tokenizer max length ", self.tokenizer.model_max_length)

    # print("target_ids arr : ", target_ids.shape)
    # print("pred_ids arr : ", pred_ids.shape)



    target_str_li = self.tokenizer.batch_decode(target_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
 
    
    # predicts what should be in the mask
    
    # get rogue scores for each string pair and avergae it
    
    # rogue score for bathc is average of that batch, loss laredy calculatd that average

    # rouge(preds, target)

    rouge_scores = self.scorer(pred_str_li, target_str_li)
    # torch.mean(torch.Tensor([self.scorer.score(target_str_li[i], pred_str_li[i])['rouge1'] for i in range(len(pred_str_li))]))
    #get f1 score of everything

    rouge1 = rouge_scores['rouge1_fmeasure']
    rouge2 = rouge_scores['rouge2_fmeasure']
    rougeL = rouge_scores['rougeL_fmeasure']
    # rougeLsum = rouge_scores['rougeLsum_fmeasure']

  

    preds_new =  torch.max(pred_ids, torch.tensor([1]).cuda())
    labels_new = torch.max(labels, torch.tensor([1]).cuda())

    # set -100 to 0's
    acc = self.acc(preds_new, labels_new)
    # self.log('training/step_acc', acc)
    # print("Target STR : ", target_str_li)
    # print("Pred   STR : ", pred_str_li)

    # print("Target FULL : ", self.tokenizer.batch_decode(target_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False))
    # print("Pred   FULL : ", self.tokenizer.batch_decode(pred_ids,skip_special_tokens=False, clean_up_tokenization_spaces=False))
    
    # print("LOSS : " +  str(loss.item()) + " , ACC: " + str(acc.item())  +" , ROUGE 1: " + str(rouge1.item())  + " , ROUGE 2: " + str(rouge2.item()) +  " , ROUGE L: " + str(rougeL.item())  )
   

    return loss, acc, rouge1, rouge2, rougeL

  def training_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)

    tensorboard_logs = {"train_loss": loss, "train_acc": acc, "train_rouge1": rouge1, "train_rouge2": rouge2, "train_rougeL": rougeL  }

    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)


    # print("TRAIN STEP: ", tensorboard_logs)

    
    return {"loss": loss, "acc": acc,   "rouge1": rouge1, "rouge2": rouge2,  "rougeL": rougeL, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    avg_train_acc = torch.stack([x["acc"] for x in outputs]).mean()
    avg_train_rouge1 = torch.stack([x["rouge1"] for x in outputs]).mean()
    avg_train_rouge2 = torch.stack([x["rouge2"] for x in outputs]).mean()
    avg_train_rougeL = torch.stack([x["rougeL"] for x in outputs]).mean()

    tensorboard_logs = {"train_loss": avg_train_loss, "train_acc": avg_train_acc, "train_rouge1": avg_train_rouge1, "train_rouge2": avg_train_rouge2, "train_rougeL": avg_train_rougeL }

    self.log('step', self.trainer.current_epoch)
    self.log('avg_train_loss', avg_train_loss)
    self.log('avg_train_acc', avg_train_acc)
    
    self.log('avg_train_rouge1', avg_train_rouge1)
    self.log('avg_train_rouge2', avg_train_rouge2)
    self.log('avg_train_rougeL', avg_train_rougeL)
    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

 
    print("TRAIN Epoch End: ", tensorboard_logs)

    # return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)
    tensorboard_logs = {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    # print("VAL STEP: ", {tensorboard_logs)
    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)

    return {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    
  def validation_epoch_end(self, outputs):
    
    avg_val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
    avg_val_rouge1 = torch.stack([x["val_rouge1"] for x in outputs]).mean()
    avg_val_rouge2 = torch.stack([x["val_rouge2"] for x in outputs]).mean()
    avg_val_rougeL = torch.stack([x["val_rougeL"] for x in outputs]).mean()

    # print("VAL EPOCH END  Keys: ", outputs.keys() )
    
    tensorboard_logs = {"val_loss": avg_val_loss, "val_acc": avg_val_acc, "val_rouge1": avg_val_rouge1, "val_rouge2": avg_val_rouge2, "val_rougeL": avg_val_rougeL}
    # return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    self.log('avg_val_loss', avg_val_loss)
    self.log('avg_val_acc', avg_val_acc)
    self.log('avg_val_rouge1', avg_val_rouge1)
    self.log('avg_val_rouge2', avg_val_rouge2)
    self.log('avg_val_rougeL', avg_val_rougeL)

    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

    print("VAL Epoch End: ", tensorboard_logs)


  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, optimizer_closure=None, 
                    on_tpu=False, using_native_amp=False, 
                    using_lbfgs=False):
    # if on_tpu == True: #self.trainer.use_tpu:
    #   # xm.optimizer_step(optimizer)
    #   optimizer.step()
    # else:
    # optimizer.step(closure=optimizer_closure)
    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams, col = "_b")
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=False, shuffle=False, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams, col = "_b")
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [ ]:
logger = logging.getLogger(__name__)



class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      output_val_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_val_results_file, "w") as writer:
        for key in sorted(metrics):
          # if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          #if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

In [ ]:
bart_args_dict = dict(
    data_dir="./gdrive/MyDrive/W266 Final Project 2/data/new_data",  #directory for all the input data files # path for data files
    output_dir="./gdrive/MyDrive/W266 Final Project 2/models/BART_base-1/", # path to save the checkpoints
    model_name_or_path='facebook/bart-base',
    tokenizer_name_or_path='facebook/bart-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu= 1,#1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    amp_backend = "apex",
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [ ]:
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', model_max_length = args_dict['max_seq_length'])

In [ ]:
# model will take in only 256 tokens max. During trianing, calling model to get prediciotns will also produce 256 tokens
# only generate function() takes in variable token length to define how many words the model should generate
# tokenizer.model_max_length

In [8]:

class SongDataset(Dataset):

# class ImdbDataset(Dataset):
  def __init__(self, tokenizer, data_dir,  type_path, max_len=256, col = ""):

    self.file_path = os.path.join(data_dir, type_path)
    self.file = self.file_path + ".csv"
    
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []
    self.col = col

    self._build()
  
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    # gets it into 1 dimension
    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
  
  def _build(self):
    self._build_examples_from_files(self.file)

  
  def _build_examples_from_files(self, file):

    # read in 1 sample
    # read in 1 input at a time

    # ierate through df rows

    text_df = pd.read_csv(self.file)
    # print("text_df : ", text_df.shape)

    for index, row  in text_df.iterrows():
      # clean strings here
      #input b for BART
      input = preprocess(row['inputs' + self.col])
      target = preprocess(row['targets' + self.col])
      
       # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input], max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors="pt"
      )

      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target], max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)



In [ ]:
# dataset = SongDataset(tokenizer, args_dict['data_dir'], 'train',  max_len=256)
# ImdbDataset(tokenizer, 'aclImdb', 'val',  max_len=512)
# len(dataset)

In [ ]:
# data = dataset[15]
# print(tokenizer.decode(data['source_ids']))
# print(tokenizer.decode(data['target_ids']))

## BART Train

In [ ]:
# args_dict.update({'data_dir': 'T5', 'output_dir': 'outputs', 'num_train_epochs':2})
bart_args = argparse.Namespace(**bart_args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=bart_args.output_dir, filename='{epoch}-{val_loss:.2f}', monitor="avg_val_loss", mode="min", save_top_k=5
)

# path to save lightning_logs
tb_logger = pl_loggers.TensorBoardLogger(save_dir=bart_args.output_dir)


bart_train_params = dict(
    logger = tb_logger,
    
    accumulate_grad_batches=bart_args.gradient_accumulation_steps,


    gpus=bart_args.n_gpu, 
    # accelerator = "cpu",

    max_epochs=bart_args.num_train_epochs,
    log_every_n_steps = 50, # change based on size of dataset
    # early_stop_callback=False,
    precision= 16 if bart_args.fp_16 else 32,
    amp_level=bart_args.opt_level,
    amp_backend = bart_args.amp_backend,
    gradient_clip_val=bart_args.max_grad_norm,
    enable_checkpointing = True,
    # checkpoint_callback=checkpoint_callback,
    callbacks=[checkpoint_callback, LoggingCallback()],
)



In [13]:
def get_dataset(tokenizer, type_path, args, col):
  return SongDataset(tokenizer=tokenizer, data_dir=args.data_dir,  type_path=type_path, max_len=args.max_seq_length, col = col)

# Fine-tune Model

In [ ]:
bart_model = BartFineTuner(bart_args)

In [ ]:
# REMOVE CPU here
bart_trainer = pl.Trainer(**bart_train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# # # clear torch cache

gc.collect()

torch.cuda.empty_cache()



In [ ]:
# start fine-tuning
bart_trainer.fit(bart_model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
Missing logger folder: ./gdrive/MyDrive/W266 Final Project/models/BART_base-K/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name   | Type                         | Params
--------------------------------------------------------
0 | model  | BartForConditionalGeneration | 139 M 
1 | acc

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


VAL Epoch End:  {'val_loss': tensor(4.4303, device='cuda:0'), 'val_acc': tensor(0.0823, device='cuda:0'), 'val_rouge1': tensor(0.2709), 'val_rouge2': tensor(0.0997), 'val_rougeL': tensor(0.2304)}


Training: 0it [00:00, ?it/s]

...

VAL Epoch End:  {'val_loss': tensor(2.8032, device='cuda:0'), 'val_acc': tensor(0.1417, device='cuda:0'), 'val_rouge1': tensor(0.5338), 'val_rouge2': tensor(0.2850), 'val_rougeL': tensor(0.4261)}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:231: UserWarning: You called `self.log('step', ...)` in your `training_epoch_end` but the value needs to be floating point. Converting it to torch.float32.
  f"You called `self.log({self.meta.name!r}, ...)` in your `{self.meta.fx}` but the value needs to"


TRAIN Epoch End:  {'train_loss': tensor(0.2113, device='cuda:0'), 'train_acc': tensor(0.1200, device='cuda:0'), 'train_rouge1': tensor(0.4029), 'train_rouge2': tensor(0.1734), 'train_rougeL': tensor(0.3110)}


Validation: 0it [00:00, ?it/s]

VAL Epoch End:  {'val_loss': tensor(2.7162, device='cuda:0'), 'val_acc': tensor(0.1447, device='cuda:0'), 'val_rouge1': tensor(0.5015), 'val_rouge2': tensor(0.2692), 'val_rougeL': tensor(0.4011)}
TRAIN Epoch End:  {'train_loss': tensor(0.1646, device='cuda:0'), 'train_acc': tensor(0.1437, device='cuda:0'), 'train_rouge1': tensor(0.4604), 'train_rouge2': tensor(0.2347), 'train_rougeL': tensor(0.3630)}


In [ ]:
# 11:10

In [ ]:
bart_args_dict['output_dir']

'./gdrive/MyDrive/W266 Final Project/models/BART_base-K/'

In [ ]:
# Save model
# CHANGE THE PATH IF NEEDED
bart_model.model.save_pretrained(bart_args_dict['output_dir'] + 'BART_base-1')
print("Saved model")

Saved model


In [ ]:
# 9:02

In [ ]:
%load_ext tensorboard
%tensorboard --logdir='./gdrive/MyDrive/W266 Final Project/models/BART_base-1/'

<IPython.core.display.Javascript object>

# Bart Base Eval

In [15]:
scorer = ROUGEScore()

accuracy = torchmetrics.Accuracy(mdmc_average = 'samplewise')


def get_test_metrics(model, tokenizer, dataloader, file_path):
  r_log = open(file_path + "rouge.txt", 'w+') #open a text file for logging
  b_log = open(file_path + "bleu.txt", 'w+') #open a text file for logging
  a_log = open(file_path + "accuracy.txt", 'w+')
  rouge1_li = []
  rouge2_li = []
  rougeL_li = []
  bleu_li = []
  accuracy_li = []
  # iterate through each batch  in dataloader
  # for batch_ind in range(len(dataloader)):
  iter_dataloader = iter(dataloader)
  # dataloader is preprocessed
  max_tokens = 100
  i = 0
  for batch in iter_dataloader:
    if i < 18:
      i+=1
      continue

    pred_ids = model.generate(input_ids=batch['source_ids'], 
                                attention_mask=batch['source_mask'],  
                                num_beams=4, #9
                                    no_repeat_ngram_size=2,
                                    num_return_sequences=1,
                          #  ,
                                    min_length = max_tokens,
                                    max_length=max_tokens
                                )

    #just get rouge metrics
    # dec = [tokenizer.decode(ids) for ids in outs]

    # get loss and accuracy 
    # list of strings 
    pred_m_ids = pred_ids[:, :max_tokens]
    pred_str = tokenizer.batch_decode(pred_m_ids, skip_special_tokens=True, 
                              clean_up_tokenization_spaces=False)
    
    clean_pred_str = [p.translate(str.maketrans('', '', string.punctuation)).replace("\n", " ") for p in pred_str ]

    # each sentence needs to be in it's own 2 brackets
    clean_pred_str_li = [p.split() for p in clean_pred_str ]

    


    # texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
    # # verses with chorus masked
    # input_str = tokenizer.batch_decode( batch['source_ids'], skip_special_tokens=True, 
    #                           clean_up_tokenization_spaces=False)
    # targets = [tokenizer.decode(ids) for ids in batch['target_ids']]
    target_m_ids =  batch['target_ids'][:, :max_tokens]
    target_str = tokenizer.batch_decode(target_m_ids, skip_special_tokens=True, 
                              clean_up_tokenization_spaces=False)
    
    clean_target_str = [p.translate(str.maketrans('', '', string.punctuation)).replace("\n", " ") for p in target_str ]
    
    clean_target_str_li = [p.split() for p in clean_target_str ]

    # accuracy
    # preds_new =  torch.max(pred_m_ids, torch.tensor([1]).cuda())
    # labels_new = torch.max(target_m_ids, torch.tensor([1]).cuda())
    # 1 accuracy across batch
    acc = accuracy(pred_m_ids, target_m_ids).item()
    accuracy_li.append(acc)


    a_s = "Batch " + str(i) + " = Acc: " +  str(acc)

    a_log.write(a_s + "\n")


    # 1 ROUGE SCORE CLACUALTE ACROSS ENTIRE BATCH
    rouge_scores = scorer(clean_pred_str, clean_target_str)
    rouge1 = rouge_scores['rouge1_fmeasure'].item()
    rouge2 = rouge_scores['rouge2_fmeasure'].item()
    rougeL = rouge_scores['rougeL_fmeasure'].item()
    # print("rougue_scores : ", rouge_scores)
    # print("rouge1 : ", rouge1)
    rouge1_li.append(rouge1)
    rouge2_li.append(rouge2)
    rougeL_li.append(rougeL)
    r_s = "Batch " + str(i) + " = Rouge 1: " + str(rouge1) + " , Rouge 2: "  + str(rouge2) + " , Rouge L: " + str(rougeL)

    # print(r_s)
    r_log.write(r_s + "\n")

    # print("clean_pred_str_li : ")
    print("PRED : ",   clean_pred_str)
    # print("LEN pred: ", len(clean_pred_str_li))
    # print("LEN pred 0: ", len(clean_pred_str_li[0]))
   
    # print("clean_target_str_li : ")
    print("TARGET : ", clean_target_str)
    # print("LEN target : ", len(clean_target_str_li))
    # print("LEN target 0: ", len(clean_target_str_li[0]))
    # 1 score for the entire batch

    batch_bleu = []
    for b_ind in range(len(clean_target_str_li)):
      b = sentence_bleu([clean_target_str_li[b_ind]], clean_pred_str_li[b_ind])
      batch_bleu.append(b)

    bleu = np.average(batch_bleu)
    bleu_li.append(bleu)
    b_s = "Batch " + str(i) + " = Bleu: " + str(bleu)
    # print(b_s)
    b_log.write(b_s + "\n") #write to log

    print("Batch " + str(i) + " = Rouge 1: " + str(rouge1) + " , Rouge 2: "  + str(rouge2) + " , Rouge L: " + str(rougeL) \
              + " , Bleu: " + str(bleu) +  " , Acc: " + str(acc))
    

    i +=1

  avg_rouge1 = np.average(rouge1_li)
  avg_rouge2 = np.average(rouge2_li)
  avg_rougeL = np.average(rougeL_li)
  avg_bleu = np.average(bleu_li)
  avg_acc = np.average(accuracy_li)

  b_log.close()
  r_log.close()
  a_log.close()

  return avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu, avg_acc




In [16]:

gc.collect()

torch.cuda.empty_cache()

In [11]:


data_args_dict = dict(
    data_dir="./gdrive/MyDrive/W266 Final Project 2/data/new_data/",  #directory for all the input data files # path for data files
    max_seq_length=256,
   
)
data_args = argparse.Namespace(**data_args_dict)


In [ ]:
# Try BART model
bart_base_dir = "./gdrive/MyDrive/W266 Final Project 2/models/BART_base-1/"
bart_base_model = BartForConditionalGeneration.from_pretrained(bart_base_dir + "BART_base-1")
bart_base_tokenizer =  BartTokenizer.from_pretrained('facebook/bart-base', model_max_length = 256)



bart_base_test_dataset = get_dataset(tokenizer=bart_base_tokenizer, type_path="test", args=data_args, col = "_b")
bart_base_dataloader = DataLoader(bart_base_test_dataset, batch_size=8, drop_last=False, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
bart_base_rouge1, bart_base_rouge2, bart_base_rougeL, bart_base_bleu, bart_base_acc = get_test_metrics(bart_base_model, bart_base_tokenizer, bart_base_dataloader, bart_base_dir + "test_logs/")


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


PRED :  ['   im a real fan and a fake groupie i dont discriminate now let me squeeze that fake booty tryna see you pop that thang like a baby uzi now back that ass up but come and serve me baby i know you listenin but ima still ask if ya heard me now peep this g shit rich gang remix your bitch wavin so much we get sea sick yeah yeah i heard you hear me', '   im a real fan and a fake groupie i dont discriminate now let me squeeze that fake booty tryna see you pop that thang like a baby uzi now back that ass up but ima still ask if ya heard me baby now peep this g shit rich gang remix your bitch wavin so much we get sea sick ya hear me yeah yeah i heard you uh got me in the zone nig', '   im a real fan and a fake groupie i dont discriminate now let me squeeze that fake booty tryna see you pop that thang like a baby uzi now back that ass up but ima still ask if ya heard me baby now peep this g shit rich gang remix your bitch wavin so much we get sea sick ya hear me yeah yeah i heard you u

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

PRED :  ['   aww ayy ima give you all of my love you know i love you but i dont want you to leave me alone so i wont let you go no ill never let ya go i cant trust her she thinks youre a sucker i had her first and then she with my brother i bought the black but then i went and changed the color to somethin crazy promise you will never see', '   i dont know what its like to be a young nigga but im not scared of the world i just want you to know that i love you more than i ever thought i would and ill never be the same again i wont ever be this different again i cant even get enough of you baby i swear to god ive been waiting for you for a while now you know i never ever been that way before i was', '   i dont know what its like to be a young nigga but im not scared of the world i just want you to know that i love you more than i ever thought i would and ill never be the same again i wont ever be this different again i cant even get enough of you baby i swear to god ive been waiting for 

In [ ]:
print("**** BART Full Test Metrics : ****")
print("Rouge 1 : ", bart_base_rouge1)
print("Rouge 2 : ", bart_base_rouge2)
print("Rouge L : ", bart_base_rougeL)
print("Bleu : ", bart_base_bleu)
print("Acc : ", bart_base_acc)

**** BART Full Test Metrics : ****
Rouge 1 :  0.16246772
Rouge 2 :  0.02335625
Rouge L :  0.10855109
Bleu :  0.00901324356546141
Acc :  0.018910605


# T5 

In [44]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [45]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    # self.hparams = hparams
    self.hparams.update(vars(hparams))
    self.save_hyperparameters()
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, model_max_length = hparams.max_seq_length )
    self.acc = torchmetrics.Accuracy(mdmc_average = 'samplewise')
    self.scorer = ROUGEScore() #rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      # doesn't have a max_length method, so it will keep trying to predict the same length as the target ids
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        # takes in labels and shifts right to create decoder input ids shift
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    # print("Output keys: ", outputs.keys())
    # print("Batch: " , batch)
    # print("Batch shape :" , batch.shape)


    # source = batch['source_ids']

    #  Pred Output size :  torch.Size([8, 256, 32128])
    # Target Labels size :  torch.Size([8, 256])


    # values are the probabilities.
    # indices are the indices
    # y_pred = y_max_idx  # predictions are really the inx \in [n_classes] with the highest scores
    
    


  
    
    # print("******")
    # print("Pred Output size : ", preds.size())
    # print("Target Labels size : ", torch.max(labels, torch.tensor([0])).size())


    pred_ids = torch.max(outputs['logits'], 2).indices

    # change pred ids index of blank string token at end to index of PAD token, like target label has
    # go backward
    for i in range(pred_ids.size(0)):
      for j in range(1, pred_ids.size(1) + 1):
        # if  value is  3, then replace current valuw ith 0 and break
        # print(k[i][j])
        if pred_ids[i][-j] == 3:
          pred_ids[i][-j]=0
        else:
          break
  

    # change all the empty strings (word ind = 3) at the end to 0's to represent PAD token ind.
    # for each tensor in pred_ids, maks only 

    pred_str_li = self.tokenizer.batch_decode(pred_ids,skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    # get the values before the cast. set -100 to 0 . Pad token or 0 is the same, it will be ignored in string
    target_ids = torch.max(batch["target_ids"], torch.tensor([0]).cuda())
    # print("target_ids : ", target_ids.size())
    # print("pred_ids : ", pred_ids.size())
    # print("Tokenizer max length ", self.tokenizer.model_max_length)

    # print("target_ids arr : ", target_ids.shape)
    # print("pred_ids arr : ", pred_ids.shape)



    target_str_li = self.tokenizer.batch_decode(target_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
 
    
    # predicts what should be in the mask
    
    # get rogue scores for each string pair and avergae it
    
    # rogue score for bathc is average of that batch, loss laredy calculatd that average

    # rouge(preds, target)

    rouge_scores = self.scorer(pred_str_li, target_str_li)
    # torch.mean(torch.Tensor([self.scorer.score(target_str_li[i], pred_str_li[i])['rouge1'] for i in range(len(pred_str_li))]))
    #get f1 score of everything

    rouge1 = rouge_scores['rouge1_fmeasure']
    rouge2 = rouge_scores['rouge2_fmeasure']
    rougeL = rouge_scores['rougeL_fmeasure']
    # rougeLsum = rouge_scores['rougeLsum_fmeasure']

  

    preds_new =  torch.max(pred_ids, torch.tensor([0]).cuda())
    labels_new = torch.max(labels, torch.tensor([0]).cuda())

    # set -100 to 0's
    acc = self.acc(preds_new, labels_new)
    # self.log('training/step_acc', acc)
    # print("Target Output : ", target_str_li)
    # print("Pred Output : ", pred_str_li)
    
    # print("LOSS", loss)
    # print("Accuracy", acc)





    return loss, acc, rouge1, rouge2, rougeL

  def training_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)

    tensorboard_logs = {"train_loss": loss, "train_acc": acc, "train_rouge1": rouge1, "train_rouge2": rouge2, "train_rougeL": rougeL  }

    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)


    # print("TRAIN STEP: ", tensorboard_logs)

    
    return {"loss": loss, "acc": acc,   "rouge1": rouge1, "rouge2": rouge2,  "rougeL": rougeL, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    avg_train_acc = torch.stack([x["acc"] for x in outputs]).mean()
    avg_train_rouge1 = torch.stack([x["rouge1"] for x in outputs]).mean()
    avg_train_rouge2 = torch.stack([x["rouge2"] for x in outputs]).mean()
    avg_train_rougeL = torch.stack([x["rougeL"] for x in outputs]).mean()

    tensorboard_logs = {"train_loss": avg_train_loss, "train_acc": avg_train_acc, "train_rouge1": avg_train_rouge1, "train_rouge2": avg_train_rouge2, "train_rougeL": avg_train_rougeL }

    self.log('step', self.trainer.current_epoch)
    self.log('avg_train_loss', avg_train_loss)
    self.log('avg_train_acc', avg_train_acc)
    
    self.log('avg_train_rouge1', avg_train_rouge1)
    self.log('avg_train_rouge2', avg_train_rouge2)
    self.log('avg_train_rougeL', avg_train_rougeL)
    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

 
    print("TRAIN Epoch End: ", tensorboard_logs)

    # return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)
    tensorboard_logs = {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    # print("VAL STEP: ", {tensorboard_logs)
    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)

    return {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    
  def validation_epoch_end(self, outputs):
    
    avg_val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
    avg_val_rouge1 = torch.stack([x["val_rouge1"] for x in outputs]).mean()
    avg_val_rouge2 = torch.stack([x["val_rouge2"] for x in outputs]).mean()
    avg_val_rougeL = torch.stack([x["val_rougeL"] for x in outputs]).mean()

    # print("VAL EPOCH END  Keys: ", outputs.keys() )
    
    tensorboard_logs = {"val_loss": avg_val_loss, "val_acc": avg_val_acc, "val_rouge1": avg_val_rouge1, "val_rouge2": avg_val_rouge2, "val_rougeL": avg_val_rougeL}
    # return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    self.log('avg_val_loss', avg_val_loss)
    self.log('avg_val_acc', avg_val_acc)
    self.log('avg_val_rouge1', avg_val_rouge1)
    self.log('avg_val_rouge2', avg_val_rouge2)
    self.log('avg_val_rougeL', avg_val_rougeL)

    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

    print("VAL Epoch End: ", tensorboard_logs)


  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, optimizer_closure=None, 
                    on_tpu=False, using_native_amp=False, 
                    using_lbfgs=False):
    # if on_tpu == True: #self.trainer.use_tpu:
    #   # xm.optimizer_step(optimizer)
    #   optimizer.step()
    # else:
    # optimizer.step(closure=optimizer_closure)
    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams, col = "")
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=False, shuffle=False, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams, col = "")
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [46]:
logger = logging.getLogger(__name__)



class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      output_val_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_val_results_file, "w") as writer:
        for key in sorted(metrics):
          # if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          #if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

In [47]:
t5_args_dict = dict(
    data_dir="./gdrive/MyDrive/W266 Final Project 2/data/new_data",  #directory for all the input data files # path for data files
    output_dir="./gdrive/MyDrive/W266 Final Project 2/models/T5_base-1/", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu= 1,#1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    amp_backend = "apex",
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [49]:
# args_dict.update({'data_dir': 'T5', 'output_dir': 'outputs', 'num_train_epochs':2})
t5_args = argparse.Namespace(**t5_args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=t5_args.output_dir, filename='{epoch}-{val_loss:.2f}', monitor="avg_val_loss", mode="min", save_top_k=5
)

# path to save lightning_logs
tb_logger = pl_loggers.TensorBoardLogger(save_dir=t5_args.output_dir)


t5_train_params = dict(
    logger = tb_logger,
    
    accumulate_grad_batches=t5_args.gradient_accumulation_steps,


    gpus=t5_args.n_gpu, 
    # accelerator = "cpu",

    max_epochs=t5_args.num_train_epochs,
    log_every_n_steps = 50, # change based on size of dataset
    # early_stop_callback=False,
    precision= 16 if t5_args.fp_16 else 32,
    amp_level=t5_args.opt_level,
    amp_backend = t5_args.amp_backend,
    gradient_clip_val=t5_args.max_grad_norm,
    enable_checkpointing = True,
    # checkpoint_callback=checkpoint_callback,
    callbacks=[checkpoint_callback, LoggingCallback()],
)



## Fine-tune T5

In [50]:
t5_model = T5FineTuner(t5_args)

In [51]:
t5_trainer = pl.Trainer(**t5_train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [62]:
# # # clear torch cache

gc.collect()

torch.cuda.empty_cache()

# # #2:14

In [ ]:
#5:21

In [54]:
# start fine-tuning
t5_trainer.fit(t5_model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
Missing logger folder: ./gdrive/MyDrive/W266 Final Project 2/models/T5_base-1/lightning_logs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/gdrive/.shortcut-targets-by-id/1tdlBf0ibenbP95uaYV-bH3xJJIgmACVX/W266 Final Project 2/models/T5_base-1 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


VAL Epoch End:  {'val_loss': tensor(6.0126, device='cuda:0'), 'val_acc': tensor(0.4800, device='cuda:0'), 'val_rouge1': tensor(0.2774), 'val_rouge2': tensor(0.1154), 'val_rougeL': tensor(0.2435)}


Training: 0it [00:00, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:287: UserWarning: The ``compute`` method of metric _ResultMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  f"The ``compute`` method of metric {self.__class__.__name__}"


VAL Epoch End:  {'val_loss': tensor(2.2618, device='cuda:0'), 'val_acc': tensor(0.5509, device='cuda:0'), 'val_rouge1': tensor(0.5510), 'val_rouge2': tensor(0.3256), 'val_rougeL': tensor(0.4806)}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:231: UserWarning: You called `self.log('step', ...)` in your `training_epoch_end` but the value needs to be floating point. Converting it to torch.float32.
  f"You called `self.log({self.meta.name!r}, ...)` in your `{self.meta.fx}` but the value needs to"


TRAIN Epoch End:  {'train_loss': tensor(0.1638, device='cuda:0'), 'train_acc': tensor(0.4459, device='cuda:0'), 'train_rouge1': tensor(0.5039), 'train_rouge2': tensor(0.2603), 'train_rougeL': tensor(0.4307)}


RuntimeError: ignored

In [55]:
t5_model

T5FineTuner(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feature

In [53]:
pd.read_csv('./gdrive/MyDrive/W266 Final Project 2/data/new_data/val.csv')

,song_id,inputs,targets,inputs_b,targets_b
0,The-roots-respond-react-lyrics,\n[Intro: Black Thought]\nIt's just - hip-hop ...,<extra_id_0>\n[Hook]\nWe settin it from Souths...,\n[Intro: Black Thought]\nIt's just - hip-hop ...,"<mask>\n[Hook]\nWe settin it from Southside, p..."
1,The-roots-respond-react-lyrics,\n[Intro: Black Thought]\nIt's just - hip-hop ...,<extra_id_0>\n[Hook x2]\nWe settin it from Sou...,\n[Intro: Black Thought]\nIt's just - hip-hop ...,<mask>\n[Hook x2]\nWe settin it from Southside...
2,The-roots-respond-react-lyrics,\n[Intro: Black Thought]\nIt's just - hip-hop ...,<extra_id_0>\n[Hook x4]\nWe settin it from Sou...,\n[Intro: Black Thought]\nIt's just - hip-hop ...,<mask>\n[Hook x4]\nWe settin it from Southside...
3,Mike-shinoda-about-you-lyrics,\n[Intro: Mike Shinoda]\nEven when it's not ab...,<extra_id_0>\n[Chorus: Mike Shinoda]\nEven whe...,\n[Intro: Mike Shinoda]\nEven when it's not ab...,<mask>\n[Chorus: Mike Shinoda]\nEven when it's...
4,Mike-shinoda-about-you-lyrics,\n[Intro: Mike Shinoda]\nEven when it's not ab...,<extra_id_0>\n[Chorus: Mike Shinoda]\nEven whe...,\n[Intro: Mike Shinoda]\nEven when it's not ab...,<mask>\n[Chorus: Mike Shinoda]\nEven when it's...
...,...,...,...,...,...
3298,Bebe-rexha-im-gonna-show-you-crazy-lyrics,\n[Verse 1]\nThere's a war inside my head\nSom...,<extra_id_0>\n[Chorus]\nAnd I'm gonna show you...,\n[Verse 1]\nThere's a war inside my head\nSom...,"<mask>\n[Chorus]\nAnd I'm gonna show you loco,..."
3299,Bebe-rexha-im-gonna-show-you-crazy-lyrics,\n[Verse 1]\nThere's a war inside my head\nSom...,"<extra_id_0>\n[Pre-Chorus]\nYeah, I'm tired of...",\n[Verse 1]\nThere's a war inside my head\nSom...,"<mask>\n[Pre-Chorus]\nYeah, I'm tired of tryna..."
3300,Bebe-rexha-im-gonna-show-you-crazy-lyrics,\n[Verse 1]\nThere's a war inside my head\nSom...,<extra_id_0>\n[Chorus]\nI'm gonna show you loc...,\n[Verse 1]\nThere's a war inside my head\nSom...,"<mask>\n[Chorus]\nI'm gonna show you loco, man..."
3301,Bebe-rexha-im-gonna-show-you-crazy-lyrics,\n[Verse 1]\nThere's a war inside my head\nSom...,"<extra_id_0>\n[Chorus]\nYeah, I'm gonna show y...",\n[Verse 1]\nThere's a war inside my head\nSom...,"<mask>\n[Chorus]\nYeah, I'm gonna show you loc..."


In [57]:
t5_model.model.save_pretrained(t5_args_dict['output_dir'] + 'T5_base-2')
print("Saved model")

Saved model


In [ ]:
t5_model

In [65]:
t5_model.model.save_pretrained('./gdrive/MyDrive/W266 Final Project 2/models/T5_base-2/T5_base-2')

## T5 LARGE Data Eval

In [17]:
# Try T5 LARGE model
t5_base_dir = "./gdrive/MyDrive/W266 Final Project 2/models/T5_base-2/"
t5_base_model = T5ForConditionalGeneration.from_pretrained(t5_base_dir + "T5_base-2")
t5_base_tokenizer =  T5Tokenizer.from_pretrained('t5-base', model_max_length = 256)


t5_base_test_dataset = get_dataset(tokenizer=t5_base_tokenizer, type_path="test", args=data_args, col = "")
t5_base_dataloader = DataLoader(t5_base_test_dataset, batch_size=8, drop_last=False, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
# 8:53
t5_base_rouge1, t5_base_rouge2, t5_base_rougeL, t5_base_bleu, t5_base_acc = get_test_metrics(t5_base_model, t5_base_tokenizer, t5_base_dataloader, t5_base_dir + "test_logs/")


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


PRED :  ['we all we got yknow lets hold it down though were gon go down this what we gotta deal with cause im a nigga and he thug team uhh yeah you know itll be hard if you dont know it til you hear me say it please daddy pay it oh', 'we all we got yknow lets hold it down though were gon go down this what we gotta deal with cause im a nigga and he thug team uhh yeah you know itll be hard if you dont know it til you hear me say it please daddy pay it oh', 'it’s nay yeah someone tell these low down and dirty fuckers who hopsin is raw nigga i used to give a lot of fucks now if you don’t know how to get your dollars up you’re gonna have to go to the shit store and get the money ooh yeah you can see the reason that you rap so violently', 'it’s nay yeah someone tell these low down and dirty fuckers who hopsin is raw nigga i used to give a lot of fucks now if you don’t know how to get your dollars up you’re gonna have to go to the shit store and get the money ooh yeah you can see the reason t

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

PRED :  ['please dont run away youre the only one whos gonna get you if im a nigga you know that hell never be the last cause you can never get away from me no never ever get out of my mind yeah just keep on letting me know and then you get back to me you got to get off my back but you never got off your back', 'ill fly away away please dont run away zoloft xanax please do you want me to go away if youre a nigga you better get em to me ooh yeah thats the way it is shit but it can be hard to get away from me so fuck with it and you can see it in my eyes', 'i aint gonna give up cause its so hard to get out of the way if youre in the mood for it then you know you need to go out the door ooh yeah you got me sucked in but you gotta get it out so you can get back out there and you have to fuck with me because you want me to be the one to', 'all i know is that my world is super slow ooh aint nothing to do but roll up po up and catch that feeling back when youre younger used to bum from my par

KeyboardInterrupt: ignored

In [ ]:
print("**** T5 Full Test Metrics : ****")
print("Rouge 1 : ", t5_base_rouge1)
print("Rouge 2 : ", t5_base_rouge2)
print("Rouge L : ", t5_base_rougeL)
print("Bleu : ", t5_base_bleu)
print("Acc : ", t5_base_acc)

In [1]:
pd

NameError: ignored

# BART Large Eval Metrics

In [38]:
bart_lg_dir ="./gdrive/MyDrive/W266 Final Project 2/models/BART_base-1/"

In [39]:
with open(bart_lg_dir + "test_logs/bart_test_metrics_1.txt", "r") as file:
    bart_test_1 = file.read()

In [40]:
met_li = [s for s in bart_test_1.split("\n")  if s.startswith("Batch")]
met_li

['Batch 0 = Rouge 1: 0.15834921598434448 , Rouge 2: 0.005998028442263603 , Rouge L: 0.09711138159036636 , Bleu: 9.41382798849399e-156 , Acc: 0.014999998733401299',
 'Batch 1 = Rouge 1: 0.20728778839111328 , Rouge 2: 0.009649274870753288 , Rouge L: 0.13660286366939545 , Bleu: 1.760427250004303e-155 , Acc: 0.00624999962747097',
 'Batch 2 = Rouge 1: 0.1764618456363678 , Rouge 2: 0.016214648261666298 , Rouge L: 0.12584653496742249 , Bleu: 1.4976845356994167e-79 , Acc: 0.004999999888241291',
 'Batch 3 = Rouge 1: 0.17718495428562164 , Rouge 2: 0.023872245103120804 , Rouge L: 0.09674724191427231 , Bleu: 2.4898241569120245e-79 , Acc: 0.027499999850988388',
 'Batch 4 = Rouge 1: 0.1517844796180725 , Rouge 2: 0.0017730495892465115 , Rouge L: 0.10735564678907394 , Bleu: 3.6451250560574115e-156 , Acc: 0.01374999899417162',
 'Batch 5 = Rouge 1: 0.12781992554664612 , Rouge 2: 0.009536662138998508 , Rouge L: 0.09865356981754303 , Bleu: 1.892573888583502e-155 , Acc: 0.0037499999161809683',
 'Batch 6 = 

In [32]:
met.split("Acc: ")[1].split(" ,")[0].replace("\\", "").replace("}", "")

'0.0037499999161809683'

In [41]:
rouge1_li = []
rouge2_li = []
rougeL_li = []
bleu_li = []
acc_li = []
for met in met_li:
  rouge1 = float(met.split("Rouge 1: ")[1].split(" ,")[0])
  rouge2 = float(met.split("Rouge 2: ")[1].split(" ,")[0])
  rougeL = float(met.split("Rouge L: ")[1].split(" ,")[0])
  bleu = float(met.split("Bleu: ")[1].split(" ,")[0])
  acc = float(met.split("Acc: ")[1].split(" ,")[0]) #.replace("\\", "").replace("}", ""))
  rouge1_li.append(rouge1)
  rouge2_li.append(rouge2)
  rougeL_li.append(rougeL)
  bleu_li.append(bleu)
  acc_li.append(acc)






In [42]:
len(rouge1_li) == len(rouge2_li) == len(rougeL_li) == len(bleu_li) == len(bleu_li)

True

In [43]:
len(rouge1_li)

372

In [44]:
len(acc_li)

372

In [45]:
print("**** BART Large Test Metrics : ****")
print("Rouge 1 : ", np.average(rouge1_li))
print("Rouge 2 : ", np.average(rouge2_li))
print("Rouge L : ", np.average(rougeL_li))
print("Bleu : ", np.average(bleu_li))
print("Acc : ", np.average(acc_li))

**** BART Large Test Metrics : ****
Rouge 1 :  0.16953404982804612
Rouge 2 :  0.02581019627033002
Rouge L :  0.11287585155717948
Bleu :  0.009506103198813082
Acc :  0.00916330604651262


# T5 Large Dataset Metrics

In [46]:
t5_lg_dir ="./gdrive/MyDrive/W266 Final Project 2/models/T5_base-2/"


In [47]:

with open(t5_lg_dir + "test_logs/t5_lg_test_metrics_1.txt", "r") as file:
    t5_test_1 = file.read()

file.close()

with open(t5_lg_dir + "test_logs/t5_lg_test_metrics_2.txt", "r") as file:
    t5_test_2 = file.read()

file.close()

In [48]:
met_li_1 = [s for s in t5_test_1.split("\n")  if s.startswith("Batch")]
met_li_1

met_li_2 = [s for s in t5_test_2.split("\n")  if s.startswith("Batch")]
met_li_2


['Batch 18 = Rouge 1: 0.17370270192623138 , Rouge 2: 0.010861342772841454 , Rouge L: 0.1047825962305069 , Bleu: 1.6637647332460733e-155 , Acc: 0.023749999701976776',
 'Batch 19 = Rouge 1: 0.1155974492430687 , Rouge 2: 0.007499999832361937 , Rouge L: 0.0903586894273758 , Bleu: 1.3236893079192239e-155 , Acc: 0.01249999925494194',
 'Batch 20 = Rouge 1: 0.1424500048160553 , Rouge 2: 0.039201341569423676 , Rouge L: 0.12389170378446579 , Bleu: 0.018608765767849426 , Acc: 0.004999999888241291',
 'Batch 21 = Rouge 1: 0.15528500080108643 , Rouge 2: 0.010691499337553978 , Rouge L: 0.09722117334604263 , Bleu: 2.0292629507027846e-155 , Acc: 0.017500000074505806',
 'Batch 22 = Rouge 1: 0.11391972750425339 , Rouge 2: 0.025477511808276176 , Rouge L: 0.08737105131149292 , Bleu: 0.014480970987888461 , Acc: 0.00624999962747097',
 'Batch 23 = Rouge 1: 0.10830868780612946 , Rouge 2: 0.017076343297958374 , Rouge L: 0.08495741337537766 , Bleu: 1.935839155980031e-155 , Acc: 0.007499999832361937',
 'Batch 24 

In [50]:

rouge1_li = []
rouge2_li = []
rougeL_li = []
bleu_li = []
acc_li = []
for met in met_li_1:
  rouge1 = float(met.split("Rouge 1: ")[1].split(" ,")[0])
  rouge2 = float(met.split("Rouge 2: ")[1].split(" ,")[0])
  rougeL = float(met.split("Rouge L: ")[1].split(" ,")[0])
  bleu = float(met.split("Bleu: ")[1].split(" ,")[0])
  acc = float(met.split("Acc: ")[1].split(" ,")[0]) 
  rouge1_li.append(rouge1)
  rouge2_li.append(rouge2)
  rougeL_li.append(rougeL)
  bleu_li.append(bleu)
  acc_li.append(acc)


for met in met_li_2:
  rouge1 = float(met.split("Rouge 1: ")[1].split(" ,")[0])
  rouge2 = float(met.split("Rouge 2: ")[1].split(" ,")[0])
  rougeL = float(met.split("Rouge L: ")[1].split(" ,")[0])
  bleu = float(met.split("Bleu: ")[1].split(" ,")[0])
  acc = float(met.split("Acc: ")[1].split(" ,")[0]) 
  rouge1_li.append(rouge1)
  rouge2_li.append(rouge2)
  rougeL_li.append(rougeL)
  bleu_li.append(bleu)
  acc_li.append(acc)





In [53]:

len(rouge1_li) == len(rouge2_li) == len(rougeL_li) == len(bleu_li) == len(acc_li)



True

In [52]:
len(rouge1_li)

35

In [54]:
len(acc_li)

35

In [55]:


print("**** T5 Large Data Test Metrics : ****")
print("Rouge 1 : ", np.average(rouge1_li))
print("Rouge 2 : ", np.average(rouge2_li))
print("Rouge L : ", np.average(rougeL_li))
print("Bleu : ", np.average(bleu_li))
print("Acc : ", np.average(acc_li))

**** T5 Large Data Test Metrics : ****
Rouge 1 :  0.14368797327790941
Rouge 2 :  0.02008058549836278
Rouge L :  0.09930300254906926
Bleu :  0.004990021627651316
Acc :  0.014785713416391186
